In [4]:
#Python standard library imports
import datetime as dt
import pathlib
import re
#Scientific Python ecosystem imports
import pandas as pd
import numpy as np
#Text mining packages
import nltk as nltk
from whoosh.lang.porter import stem
#MM Import
import datetime as dt
import pathlib
import logging
import copy
import unicodedata

from docx2python import docx2python
from bs4 import BeautifulSoup
import html2text
import urllib.request

from polmap.polmap import preprocess_text, doc2text # replaced the keyword processing block

from collections import OrderedDict

In [5]:
def prepare_keywords(a_string, stop_words, exception_dict=None, regex_dict=None):
    """
    Prepare text for mapping.
    """
    text_string = a_string

    if text_string==None: #this should be moved to the prepare keywords wrapper function
        return None
    # if text_string is not str:
    #     raise TypeError('text_string is not a string') 
    #     #How to return the name of the variable passed by user with format?
    # Get error when using it with apply and lambda in pandas
    
    if exception_dict==None:
        exception_dict = {"aids": "ai&ds&",
                          "productivity": "pro&ductivity&",
                          "remittances" : "remit&tance&"                 
                          }
    elif exception_dict is not dict:
        raise TypeError('exception_dict is not of type dict')
    

    if regex_dict == None:
        regex_dict = OrderedDict([(r'[^a-zA-Z-. ]+', ''), (r'([\w-]+)', r' \1 ')])
    elif not isinstance(regex_dict, OrderedDict):
        raise TypeError('regex_dict is not of type Ordered dict')
        

    #remove all from stop_words to keep in keywords.
    # Review scoping rules in python, this fails with:
    # NameError: name 'stop_words' is not defined when called in lambda function
    # I would expect the variable to always exist whenever calling the function, but it does not.
    #if stop_words==None:
    #    stop_words = set(nltk.corpus.stopwords.words("english"))
    #    stop_words.remove("all")

       
    #text_string = text_string.replace('\xa0',' ') #Remove some weird \xa0 characters

    text_string = text_string.lower().strip()

    for pattern, substitution in regex_dict.items():
        text_string = re.sub(pattern, substitution, text_string)
    
    #text_string = re.sub(r'([\w-]+)', r' \1 ', text_string) #Equivalent to center, adds leading and trailing space to the captured group
    
    text_string = text_string.replace(' rd ', ' R&D ')
    
    text_string = re.sub(r'([a-zA-z-]{3,}|ph)', r'\1', text_string)
    
    # not sure this is working the way intended, 
    # if the plan was to drop two characters words,
    # it is not  as we are however counting also spaces.
    # an easy fix would be to move it before the centering of the terms
    
    for key, value in exception_dict.items(): #Protect exceptions from stemming
        text_string = text_string.replace(key, value)
    
    for word in stop_words: #Remove stopwords
        text_string = text_string.replace(' '+word+' ', '') 
    
    text_string = re.sub(r'[a-zA-z&-]+', #Find words wirth regex. It can be improved by capturing pattern between word boundaries.
    lambda rgx_word: ' '+stem(rgx_word.group())+' ', #Stem words, however stemming is skipped if string contains space.
    text_string)
        
    for key, value in exception_dict.items(): #Restore words from exception protection
        text_string = text_string.replace(value, key)
    
    text_string = ' '+text_string+' '
    
    text_string = re.sub(r' {2,}', r' ', text_string)
        
    return text_string

In [20]:
######################################
########### 2) MM Read the list of keywords and apply the prepare_keyords text processing function from polmap

keys = pd.read_excel('keys_update_27012020.xlsx', sheet_name = 'Target_keys' ) #MM 'keys_from_RAKE-GBV_DB_SB_v3.xlsx', sheet_name= 'Sheet1'

stop_words = set(nltk.corpus.stopwords.words("english"))
stop_words.remove("all")

raw_keywords_df=keys['Keys'].str.split(';', expand=True)

keys['Keys']=keys['Keys'].apply(lambda keywords: [preprocess_text(keyword, stop_words) for keyword in keywords.split(';')[:-1]])




In [21]:
keys['Keys'][0]

[' intern poverti line ',
 ' extrem poverti ',
 ' abject poverti ',
 ' absolut poverti ',
 ' destitut ',
 ' penuri ',
 ' sever depriv ',
 ' help peopl suffer food materi depriv ',
 ' extrem poor ',
 ' incid poverti ',
 ' household poverti ']

In [7]:
print(stop_words)

{'herself', 't', 'needn', 'your', 'aren', 'between', 'our', 'yourself', 'i', "wasn't", 'by', 'some', 'if', 'but', "hadn't", 'into', 'o', "isn't", 'this', "you've", 'a', 'them', 'him', 'through', 'under', 'or', 'were', 'having', 'there', 'is', 're', 'they', 'once', 'who', 'he', 'to', 'are', 'she', 'own', 'can', 'at', 'has', 'y', 'doesn', 'above', 'out', "shan't", 'which', "doesn't", "she's", 'because', 'very', 'these', 'll', 'so', 'until', 'as', "should've", "you'll", 'here', 'don', 'my', 'himself', 's', 'been', 'itself', "haven't", 'doing', 'his', "don't", 'yours', 'weren', "didn't", 'its', 'do', 'no', 'what', 'when', 'myself', 'again', 'couldn', "needn't", 'won', 'against', 'why', 'ain', 'her', 'same', 'mightn', 'isn', 'ourselves', 'ours', 'most', 'wouldn', 'have', 'haven', 'more', 'be', 'over', 'before', 'me', "it's", 'the', 'wasn', 'hasn', 'it', 'shan', 'off', 'now', 've', 'whom', 'we', 'not', 'up', 'that', 'themselves', 'on', 'was', 'an', 'below', 'ma', "that'll", 'other', "mightn'

In [8]:
keywords = keys['Keys'].apply(pd.Series)

keywords.replace(to_replace={'':None, np.NaN:None}, inplace=True)


In [9]:
keywords.head()

,0,1,2,3,4,5,6,7,8,9,...,1909,1910,1911,1912,1913,1914,1915,1916,1917,1918
0,,n,,e,r,n,,,,,...,None,None,None,None,None,None,None,None,None,None
1,,e,v,e,r,e,l,,,,...,None,None,None,None,None,None,None,None,None,None
2,,,c,,,l,,r,,,...,None,None,None,None,None,None,None,None,None,None
3,b,,,,c,,,e,r,v,...,None,None,None,None,None,None,None,None,None,None
4,,,,,,,e,r,,r,...,None,None,None,None,None,None,None,None,None,None


In [10]:
raw_keywords_df.replace(to_replace={'':None, np.NaN:None}, inplace=True)
raw_keywords_df.drop(labels=57, axis=1, inplace=True)


In [11]:

processed_keywords_df=raw_keywords_df.applymap(lambda x: prepare_keywords(x, stop_words))
processed_keywords_df.replace(to_replace={'':None, np.NaN:None}, inplace=True)


In [12]:
new_vs_standard=processed_keywords_df.compare(keywords, keep_shape=True)
raw_vs_new = raw_keywords_df.compare(processed_keywords_df, keep_shape=True)
raw_vs_standard = raw_keywords_df.compare(keywords, keep_shape=True)
#comparison[1][1]='=IF(AND(ISBLANK(New_preprocessing!B2),ISBLANK(Standard_processing!B2)),"",IF(New_preprocessing!C2=Standard_preprocessing!C2,TRUE,FALSE))'


ValueError: Can only compare identically-labeled DataFrame objects

In [ ]:
keywords.head()

In [ ]:
processed_keywords_df.head()

In [ ]:
raw_keywords_df.head()

In [ ]:
with pd.ExcelWriter('updated_processed_keywords_test.xlsx', engine='xlsxwriter') as writer:
    raw_keywords_df.to_excel(writer, sheet_name='Raw_keywords')
    processed_keywords_df.to_excel(writer, sheet_name='New_preprocessing')
    keywords.to_excel(writer, sheet_name='Standard_preprocessing')
    new_vs_standard.to_excel(writer, sheet_name='New_vs_Standard')
    raw_vs_new.to_excel(writer, sheet_name='Raw_vs_New')
    raw_vs_standard.to_excel(writer, sheet_name='Raw_vs_Standard')


In [ ]:
regex_dict = OrderedDict([(r'[^a-zA-Z- ]+', ''), (r'([\w-]+)', r' \1 ')])

In [ ]:
for key, value in regex_dict.items():
    print(key, value)

In [ ]:
regex_dict is OrderedDict

In [ ]:
not isinstance(regex_dict, OrderedDict)